<a href="https://colab.research.google.com/github/fabiomatricardi/LaMiniPower/blob/main/Study_of_Summarization_with_FlanT5LaMini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://mbzuai-nlp.github.io/LaMini-LM/images/LaMini-LM-solo.png" height=300>


# LaMini power: when a small guy can beat the Giants

# Summarize YouTube subs  and text with_FlanT5LaMini


---



---



Using Page Plain Text Chrome addon to download plain text from the Medium article 
Load it on Google Colab
Use FlanT5-LaMini for Summarization and chat?

## TOC

>[Summarize YouTube subs  and text with_FlanT5LaMini](#scrollTo=Z4e_7ABOMoB3)

>>[TOC](#scrollTo=m1_Txof1VVTl)

>>[Install dependencies](#scrollTo=BAMCzEQ1VVkZ)

>>[Prepare Models](#scrollTo=CgdXnNJBJ9Hd)

>>>[Get HUGGINGFACEHUB_API_KEY](#scrollTo=3LKFoMvsKL0b)

>>>[click on this link for only TEXT summarization](#scrollTo=stxfhSbXGalD)

>>>[Youtube Summarization](#scrollTo=IKwe-n9KHU1g)

>>>[Instructions](#scrollTo=HnZ9NS0IKh1O)

>>[method for documents](#scrollTo=7CXaaYR6JhNq)

>>[Decoder Only Models](#scrollTo=XPkZd9PH_uVI)

>[Summarize from Copy Paste](#scrollTo=GXArZj0J_dAS)

>>>[Add a spinner duiring summary generation](#scrollTo=IqtGAubMpPkl)

>>[The Winner is... 🤗 MBZUAI/LaMini-Flan-T5-248M](#scrollTo=9_xFwoQKAuKN)

>[Test with locally downloaded model](#scrollTo=AhO7RfFStfLP)

>>>[Download Model from HuggingFace MBZUAI/LaMini-Flan-T5-248M and move it to Model folder](#scrollTo=MMToFuhdMdkL)

>>[Import libraries](#scrollTo=w4PdoHUYBw52)

>>[Iterate on the chunks, summarize one by one, and then summarize the summary](#scrollTo=0-qJvLLBU_iX)

>>[Using LangChain CHAIN](#scrollTo=K45BGKhBfniS)

>>>[The 🥇 text2text generation pipeline with   🦜🔗 LangChain win both in terms of time ⏲ and in terms of 👍 quality](#scrollTo=KBI463bjwFrT)



## Install dependencies

Install pytube and apply patch for captions and cypher

In [1]:
%%capture
!pip install pytube==12.1.3
!git clone https://github.com/fabiomatricardi/pytubeFix.git
%shell cp '/content/pytubeFix/1213-python10/captions.py' '/usr/local/lib/python3.10/dist-packages/pytube/captions.py'
%shell cp '/content/pytubeFix/1213-python10/cipher.py' '/usr/local/lib/python3.10/dist-packages/pytube/cipher.py'

In [2]:
%%capture
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]
!pip install rich

In [3]:
#@title Restart Runtime {display-mode: "form"}
import ipywidgets as widgets
def restart(b):
  exit()

button2 = widgets.Button(
    description='Restart Runtime',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button2.on_click(restart)
button2


Button(button_style='warning', description='Restart Runtime', icon='check', style=ButtonStyle(), tooltip='Clic…

## Prepare Models

### Get HUGGINGFACEHUB_API_KEY

In [5]:
import os
import requests
# PUT HERE YOUR HUGGING FACE API TOKEN shuold start with hf_XXXXXXX...
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_XXXXXXXX"

from langchain.document_loaders import TextLoader  #for textfiles
from langchain.text_splitter import CharacterTextSplitter #text splitter
from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace models
from langchain.vectorstores import FAISS  #facebook vectorizationfrom langchain.chains.question_answering import load_qa_chain
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

If you want TEXT summarization, not from youtube<br>
### [click on this link for only TEXT summarization](#scrollTo=AhO7RfFStfLP&uniqifier=1) 

### Youtube Summarization

In [2]:
#################### VIDEO AND SCRIPT SECTION #############################
# Extract Video Informations for future use in PDF or word processor
# Prepare the text for Summarization (no fuss, plain text)
###########################################################################
import ssl
from pytube import YouTube as YT
import re
import textwrap

# SSL for proxied internet access
ssl._create_default_https_context = ssl._create_unverified_context

# paste here the url of the youtubevideo you want
url = "https://youtu.be/Xxxuw4_iCzw" # or https://youtu.be/5g1z4Sr-UHM 
# we instantiate a YouTube object already with our 
myvideo = YT(url, use_oauth=True, allow_oauth_cache=True)
# required only for the first time to know what languages are aavailable
print(myvideo.title)
print(myvideo.captions) #print the options of languages available
code = input("input the code you want: ")
print("Scraping subtitles...")
sub = myvideo.captions[code]
caption = sub.generate_srt_captions()
#print(caption)

# Club Video Title, details and Description, only for printed version
# not for the Sumarization one
# possible in future to prepare for Markdown to PDF export
import datetime
m1 = f"TITLE: {myvideo.title}"+'\n'
m2 = f"thumbnail url: {myvideo.thumbnail_url}"+'\n'
m4 = f"video Duration: {str(datetime.timedelta(seconds=myvideo.length))}"+'\n'
m5 = "----------------------------------------"+'\n'
#m6 = textwrap.fill(myvideo.description, 80)+'\n'  #solution not good
m6 = myvideo.description+'\n'
m7 = "----------------------------------------"+'\n'
m_intro = m1+m2+m4+m5+m6+m7

# Function to clean up the srt text
def clean_sub(sub_list):
    lines = sub_list
    text = ''
    for line in lines:
        if re.search('^[0-9]+$', line) is None and re.search('^[0-9]{2}:[0-9]{2}:[0-9]{2}', line) is None and re.search('^$', line) is None:
            text += ' ' + line.rstrip('\n')
        text = text.lstrip()
    #print(text)
    return text

print("Transform subtitles to TEXT...")
srt_list = str(caption).split('\n')  #generate a list with all lines
final_text = clean_sub(srt_list)
#print(final_text)
#PREPARE A LONG STRING FOR THE SUMMARIZATION, no Video Description Here
intro_summarization = 'Video Title: '+myvideo.title+' - (video url: )'+url+' ---  '+'\n'
#PREPARE A LONG TEXT with details and Video Description Here: to be used
#as a note or Blog
intro_blog = 'Video Title: '+myvideo.title+'\n'+'Video url: '+url+'\n'+'-------------'+'\n'
summarization_text = intro_summarization + final_text
import textwrap
blog_text = m_intro + textwrap.fill(final_text, 70)

def correct_filename_cr(title):
  string = title
  finalfilename = ''.join(e for e in string if e.isalnum())+'_cr.txt'
  return finalfilename
def correct_filename_nocr(title):
  string = title
  finalfilename = ''.join(e for e in string if e.isalnum())+'_nocr.txt'
  return finalfilename

print("Saving to TXT file...")
filename_file = correct_filename_cr(myvideo.title) #with carriage return
filename_summary = correct_filename_nocr(myvideo.title) #with no carriege return

#prepare the text for the Blog or Notes
#with carriage return. We want cool text only
#for the report.
import textwrap
wrapped_text = textwrap.fill(final_text, 100)

# write the cleaned up text into 2 file2
# One with carriage returns, one withouth them
with open(filename_file, 'w') as f:
    f.write(blog_text)
f.close()
with open(filename_summary, 'w') as f:
    f.write(summarization_text)
f.close()

Please open https://www.google.com/device and input code HJBZ-KZWM
Press enter when you have completed this step.
LangChain: Run Language Models Locally - Hugging Face Models
{'a.en': <Caption lang="English (auto-generated)" code="a.en">}
input the code you want: a.en
Scraping subtitles...
Transform subtitles to TEXT...
Saving to TXT file...


### Instructions
summarization_text is in the format for the Summarization Pipeline
<br>
blog_text is in the format for text processing or future studies
> It may good to include also the summarization in the beginning of the text but it will more likely mess up with the vector indexing
 

In [2]:
import rich
from rich.console import Console
from rich.panel import Panel
from rich import print

########## INITIALIZE RICH CONSOLE  ##################
console = Console()

In [3]:
# Document Loader
from langchain.document_loaders import TextLoader
# text splitter for create chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
# FAISS  library for similaarity search
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text
#print(wrap_text_preserve_newlines(str(documents[0])))   

In [ ]:
# documents  #if youused above and want to see the result
# added on 17 May 2023
# Summarization is done with the CHAIN, not with the pipeline
#-----------------------------------------------
from langchain.chains.summarize import load_summarize_chain
llm9 = HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-248M", model_kwargs={"temperature":0, "max_length":512})
llm11 = HuggingFaceHub(repo_id="MBZUAI/LaMini-T5-223M", model_kwargs={"temperature":0, "max_length":512})

def summarize_docs(textfile, llm):
    """
    takes as argument the textfile name : str
    and the llm (in this case from HuggingFaceHub)
    To be explored how to do it locally for the pipeline...
    """
    summaries = []
    loader = TextLoader(textfile)
    docs = loader.load_and_split()
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary = chain.run(docs)
    print("Summary for: ", textfile)
    print(summary)
    print("\n")
    summaries.append(summary)
    
    return summaries



## method for documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
# HERE 2 MODELS to be used with HuggingFace API for Summarization
llm9 = HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-248M", model_kwargs={"temperature":0, "max_length":512})
llm11 = HuggingFaceHub(repo_id="MBZUAI/LaMini-T5-223M", model_kwargs={"temperature":0, "max_length":512})
# Define Function that returns Summary of textfile
def summarize_docs(textfile, llm):
    """
    takes as argument the textfile name : str
    and the llm (in this case from HuggingFaceHub)
    To be explored how to do it locally for the pipeline...
    """
    summaries = []
    loader = TextLoader(textfile)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = 300,
        chunk_overlap  = 20,
    )
    texts = text_splitter.create_documents(docs)
    print(texts[0])
    print(texts[1])
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary = chain.run(texts)
    print("Summary for: ", textfile)
    print(summary)
    print("\n")
    summaries.append(summary)
    
    return summaries



In [7]:
llm6=HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-783M", model_kwargs={"temperature":0, "max_length":512})
llm2=HuggingFaceHub(repo_id="declare-lab/flan-alpaca-large", model_kwargs={"temperature":0, "max_length":512})
llm9 = HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-248M", model_kwargs={"temperature":0, "max_length":512})
llm11 = HuggingFaceHub(repo_id="MBZUAI/LaMini-T5-223M", model_kwargs={"temperature":0, "max_length":512})
llm8 = HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-77M", model_kwargs={"temperature":0, "max_length":512})
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
#fname = 'yourtextfile'
fname = '/content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt'
with open(fname) as f:
    doc = f.read()
summaries = []
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 900,
    chunk_overlap  = 30,
    length_function = len,
)
texts = text_splitter.create_documents([doc])

In [11]:
from langchain.chains.summarize import load_summarize_chain

In [15]:
# open the text file for summarization, without carriage return
# and split into chunks
fname = '/content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt'
with open(fname) as f:
    doc = f.read()
f.close() #always remember to close!
summaries = []
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.create_documents([doc])

# Call the Hugging Face API to run the Summarization Chain
llm_id = 'MBZUAI/LaMini-T5-223M'
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)
summaries.append(summary)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
# calculate the elapsed time and print to a Rich Console
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing Summarization Chain

Token indices sequence length is longer than the specified maximum sequence length for this model (1453 > 1024). Running this sequence through the model will result in indexing errors


[Bold green]Summary for:  /content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt

 with MBZUAI/LaMini-T5-223M 

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The video shows how to use language models and hugging face models in apps, including using link chain,         │
│ download models, and use hugging face pipelines. The model can be used locally using a lag chain, and the model │
│ can be pre-trained for a specific type of model. The model can be used locally using a link chain, and the      │
│ model can be used locally using a Google collab notebook. The video also explains how to use a text-to-text     │
│ generation model and a decoder-only model using a local large. The model can be pre-trained for a specific      │
│ model and can be used locally using a link chain. The video also provides a link to the video and a link to the │
│ video.                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:42.928109

In [18]:
# Call the Hugging Face API to run the Summarization Chain
# here we will use a bigger model, the LaMini-Flan-T5-783M
llm_id = 'MBZUAI/LaMini-Flan-T5-783M'
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing Summarization Chain

Token indices sequence length is longer than the specified maximum sequence length for this model (1331 > 1024). Running this sequence through the model will result in indexing errors


Summary for:  /content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt

 with MBZUAI/LaMini-Flan-T5-783M 

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The article explains how to use Collage language models locally through API calls using the "packing phase" and │
│ "length chain" platforms. It explains how to define a chain using a prompt template and how to use the pipeline │
│ for local LN and prediction. The article also provides a crash course on the video and encourages comments. The │
│ video is available for free on the Google Play Store.                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:02:26.269167

---

In [19]:
# Using the smallest FlanT5 model
llm_id = 'MBZUAI/LaMini-Flan-T5-77M'
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing Summarization Chain

Token indices sequence length is longer than the specified maximum sequence length for this model (1497 > 1024). Running this sequence through the model will result in indexing errors


Summary for:  /content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt

 with MBZUAI/LaMini-Flan-T5-77M 

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The article explains how to use the LangChain: Run Language Models Locally and Hugging Face Models, which       │
│ showcases the local language model and bigging face model. The package phase provides open source models that   │
│ can be used in your own applications. The length chain is a powerful platform that allows you to build apps     │
│ based on larger language models. The first option is to use collage language models from hugging face with link │
│ chain the first. The second option is to download all these models locally and then use Hugging Face pipelines  │
│ to interact with these models. The first option is to download all these models locally and then use Hugging    │
│ Face pipelines to interact with these models. The second option is to download all these models locally and     │
│ then use Hugging Face pipelines to interact with these models. The game phase API is expected to be able to run │
│ the models locally using hugging face pipelines. The prompt template is a template that allows users to provide │
│ specific variables to a model, and the prompt template is a template that allows users to pass on the prompt    │
│ and what question they are expecting. The game phase API is expected to be able to run the models locally using │
│ hugging face pipelines. The game phase API is expected to be able to run the models locally using hugging face  │
│ pipelines. The prompt template is a template that allows users to pass on the prompt and what question they are │
│ expecting. The prompt template is a template that allows users to pass on the prompt and what question they are │
│ expecting. The model ID is used to identify the local uh L M using a pre-trained model for causal Ln. The model │
│ ID is used to identify the local uh L M using the pipeline. The model ID is used to identify the local uh L M   │
│ using the pipeline. The video provides instructions on how to describe a random chain using a hugging face      │
│ pipeline from a lag chain using the prompt and local telem. The video provides a crash course on the algorithm  │
│ to check the next video on like chain.                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:50.393153

In [ ]:
import datetime
llm_id = 'declare-lab/flan-alpaca-large'
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1224 > 1024). Running this sequence through the model will result in indexing errors


Summary for:  /content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt
with declare-lab/flan-alpaca-large 

-----------------------------------
This video demonstrates how to use hugging face models to generate natural language models with minimal
computational resources. It demonstrates how to use the API to access models, how to use hugging face
pipelines to generate natural language models, and how to use a text generation model to generate text from a
prompt.
-----------------------------------
All indexing completed in 0:00:21.820118


In [20]:
llm_id = 'MBZUAI/LaMini-T5-223M'
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing Summarization Chain

Token indices sequence length is longer than the specified maximum sequence length for this model (1453 > 1024). Running this sequence through the model will result in indexing errors


Summary for:  /content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt

 with MBZUAI/LaMini-T5-223M 

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The video shows how to use language models and hugging face models in apps, including using link chain,         │
│ download models, and use hugging face pipelines. The model can be used locally using a lag chain, and the model │
│ can be pre-trained for a specific type of model. The model can be used locally using a link chain, and the      │
│ model can be used locally using a Google collab notebook. The video also explains how to use a text-to-text     │
│ generation model and a decoder-only model using a local large. The model can be pre-trained for a specific      │
│ model and can be used locally using a link chain. The video also provides a link to the video and a link to the │
│ video.                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:01.630381

In [23]:
# Using the medium FlanT5 model
llm_id = 'MBZUAI/LaMini-Flan-T5-248M'
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing Summarization Chain

Token indices sequence length is longer than the specified maximum sequence length for this model (1401 > 1024). Running this sequence through the model will result in indexing errors


Summary for:  /content/LangChainRunLanguageModelsLocallyHuggingFaceModels_nocr.txt

 with MBZUAI/LaMini-Flan-T5-248M 

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The video explains how to run language models locally using Hugging Face with link chain, which allows for      │
│ access to local and locally-run models through API calls and Transformers. The article also provides            │
│ instructions on how to fine-tune local or locally-run GPU models in a lag chain based on the Google collab      │
│ notebook. The team is analyzing the quality of the model used through the game phase API and identifying ways   │
│ to run locally using hugging face pipelines. The video provides a crash course on Lang chain and recommends     │
│ watching the next video on Like Chain.                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:44.993300

## Decoder Only Models

In [28]:
llm_id = 'MBZUAI/LaMini-GPT-1.5B' #'MBZUAI/LaMini-Neo-125M' API non funziona
import datetime
summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarization Chain")

# Call the Chain and run it
# Decoder Only models cannot have more than 500 as max lenght
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0.2, "max_length":500})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)

# print the results with rich text format
console.print("[bold green]Summary for: ", fname)
console.print(f"[italic black] with {llm_id} \n")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(Panel(str(summaries[0]), title='AI Summarization'))
console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing Summarization Chain

ValueError: ignored

`Raised API error, but I am not sure if this is because of the Decoder-only model`

In [ ]:
llm_id = 'MBZUAI/LaMini-Cerebras-111M'  #'MBZUAI/LaMini-GPT-124M' not working API
summaries = []
import datetime
start = datetime.datetime.now() #not used now but useful
#####################################################################################
#               for TEXT-GENERATION models max_lenght CANNOT be > than 500          #
#####################################################################################
llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0.5, "max_length":480})
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(texts)
print("Summary for: ", fname)
print(f"with {llm_id} \n")
print("-----------------------------------")
summaries.append(summary)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start  
print(wrap_text_preserve_newlines(str(summaries[0])))
print("-----------------------------------")
print(f"All indexing completed in {delta}")

`Raised API error, but I am not sure if this is because of the Decoder-only model`



---



---



---



# Summarize from Copy Paste

In [32]:
import os
import requests
import datetime
import ipywidgets as widgets
from IPython.display import Image, display
from tqdm import tqdm

global testo 
testo = widgets.Textarea(
                        value='',
                        placeholder=f'Paste here the text you want to summarize',
                        description=f'TEXT:',
                        disabled=False,
                        rows = 20,   
                        layout = widgets.Layout( width='75%')
                            )
# Club text spitting and Summarization Chain into a Function
# on the widgets a (b) parameter is mandatory, even if you
# don't pass any variable.
#-----------------------------------------------------------
def btn_summarization(b):
      text_splitter = RecursiveCharacterTextSplitter(
          # Set a really small chunk size, just to show.
          chunk_size = 800,
          chunk_overlap  = 30,
          length_function = len,
          )
      texts = text_splitter.create_documents([testo.value])
      llm_id = 'MBZUAI/LaMini-Flan-T5-248M'
      fname = "From Copy/Paste"
      import datetime
      summaries = []
      start = datetime.datetime.now() #not used now but useful
      llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
      chain = load_summarize_chain(llm, chain_type="map_reduce")
      summary = chain.run(texts)
      # print the results with rich text format
      console.print("[bold green]Summary for: ", fname)
      console.print(f"[italic black] with {llm_id} \n")
      summaries.append(summary)
      stop = datetime.datetime.now() #not used now but useful  
      delta = stop-start  
      print(Panel(str(summaries[0]), title='AI Summarization'))
      console.print(f"[red bold]Summarization completed in {delta}")

#--------Define a pushButton and associate Click to the Function-------
btn_sum = widgets.Button(
    description='Start Summarization',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_sum.on_click(btn_summarization)
#--------------------Display the 2 elements------------------------
display(testo)
display(btn_sum)


Textarea(value='', description='TEXT:', layout=Layout(width='75%'), placeholder='Paste here the text you want …

Button(button_style='info', description='Start Summarization', icon='check', style=ButtonStyle(), tooltip='Cli…

Token indices sequence length is longer than the specified maximum sequence length for this model (1249 > 1024). Running this sequence through the model will result in indexing errors


Summary for:  From Copy/Paste

 with MBZUAI/LaMini-Flan-T5-248M 

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The article discusses how to use LLMs on domain-specific knowledge base using Retrieval-Augmented Generation,   │
│ which involves presenting LLMs with relevant information and allowing them to ask questions about the internal  │
│ knowledge base. The article also discusses how to fine-tune a bookworm using a fine-tuned LLM and how it can be │
│ used to unlock information stored in a specific knowledge base. The article also discusses the importance of    │
│ fine-tuning LLMs and how it works best if the specific knowledge base does not contain super specific jargon.   │
│ The article also discusses the challenges of setting up a successful RAG solution, including integrating APIs   │
│ and generative capabilities, increasing traction, and addressing the potential for mediocre versions. The       │
│ article also suggests looking for a RAG-based solution that offers specific knowledge and business value, and   │
│ suggests looking for a demo solution to help government staff answer parliamentary questions more easily using  │
│ technology.                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:59.921515

### Add a spinner duiring summary generation

In [19]:
#@title
import os
import requests
import datetime
import ipywidgets as widgets
from IPython.display import Image, display
from tqdm import tqdm

global testo 
testo = widgets.Textarea(
                        value='',
                        placeholder=f'Paste here the text you want to summarize',
                        description=f'TEXT:',
                        disabled=False,
                        rows = 20,   
                        layout = widgets.Layout( width='75%')
                            )
# Club text spitting and Summarization Chain into a Function
# on the widgets a (b) parameter is mandatory, even if you
# don't pass any variable.
# Add a Spinner to monitor the progress ongoing
#-----------------------------------------------------------
def btn_summarization(b):
      with console.status('Running Summarization pipeline...', spinner='material'):
        text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size = 700,
            chunk_overlap  = 30,
            length_function = len,
            )
        texts = text_splitter.create_documents([testo.value])
        llm_id = 'MBZUAI/LaMini-Flan-T5-248M'
        fname = "From Copy/Paste"
        import datetime
        summaries = []
        start = datetime.datetime.now() #not used now but useful
        llm=HuggingFaceHub(repo_id=llm_id, model_kwargs={"temperature":0, "max_length":512})
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(texts)
        # print the results with rich text format
        console.print("[bold green]Summary for: ", fname)
        console.print(f"[italic black] with {llm_id} \n")
        summaries.append(summary)
        stop = datetime.datetime.now() #not used now but useful  
        delta = stop-start  
        print(Panel(str(summaries[0]), title='AI Summarization'))
        console.print(f"[red bold]Summarization completed in {delta}")

        btn_dwn = widgets.Button(
                                  description='Download Summary',
                                  disabled=False,
                                  button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                                  tooltip='Click me',
                                  icon='check' # (FontAwesome names without the `fa-` prefix)
                              )
        def download_sum(b):
          with open('generated_summary.txt', 'w') as f:
            f.write(str(summaries[0]))
          f.close()
          console.print("[bold red blink]File Downloaded into Drive")
          
        btn_dwn.on_click(download_sum)
        display(btn_dwn)
        

#--------Define a pushButton and associate Click to the Function-------
btn_sum = widgets.Button(
    description='Start Summarization',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_sum.on_click(btn_summarization)
#--------------------Display the 2 elements------------------------
display(testo)
display(btn_sum)

Textarea(value='', description='TEXT:', layout=Layout(width='75%'), placeholder='Paste here the text you want …

Button(button_style='info', description='Start Summarization', icon='check', style=ButtonStyle(), tooltip='Cli…

Output()

Token indices sequence length is longer than the specified maximum sequence length for this model (1496 > 1024). 
Running this sequence through the model will result in indexing errors

Summary for:  From Copy/Paste

 with MBZUAI/LaMini-Flan-T5-248M 

╭────────────────────────────── AI Summarization ──────────────────────────────╮
│ The article discusses how to leverage LLMs on a domain-specific knowledge    │
│ base, including using RAG to Riches to talk to data, Retrieval-Augmented     │
│ Generation to unlock information stored in a specific knowledge base, and    │
│ how to fine-tune LLMs to follow the tone and lingo present in the knowledge  │
│ base. The article also discusses the challenges of setting up a successful   │
│ RAG solution, including avoiding hallucination and ensuring that LLM         │
│ solutions are kept in production. The article also suggests exploring an     │
│ RAG-based solution if interested in the concept of Retrieval-Augmented       │
│ Generation, and demonstrating the effectiveness of RAG in answering          │
│ parliamentary questions.                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯


Summarization completed in 0:00:01.102382

Button(button_style='danger', description='Download Summary', icon='check', style=ButtonStyle(), tooltip='Clic…

File Downloaded into Drive

## The Winner is... 🤗 MBZUAI/LaMini-Flan-T5-248M

# Test with locally downloaded model

If you want to try YoutubeSummarization [click here](#scrollTo=IKwe-n9KHU1g&uniqifier=1)

In [20]:
%%capture
!pip install mkl mkl-include
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
!pip install git+https://github.com//huggingface/transformers 
!pip install accelerate
!pip install rich

In [ ]:
#@title Restart Runtime {display-mode: "form"}
import ipywidgets as widgets
def restart(b):
  exit()

button2 = widgets.Button(
    description='Restart Runtime',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button2.on_click(restart)
button2


Button(button_style='warning', description='Restart Runtime', icon='check', style=ButtonStyle(), tooltip='Clic…

### Download Model from HuggingFace *MBZUAI/LaMini-Flan-T5-248M* and move it to Model folder

In [1]:
%%capture
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/.gitattributes
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/.gitignore
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/README.md
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/config.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/generation_config.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/pytorch_model.bin
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/special_tokens_map.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/spiece.model
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/tokenizer.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/tokenizer_config.json
!wget https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M/resolve/main/training_args.bin
!mkdir model
!mv /content/.gitattributes /content/model/.gitattributes
!mv /content/.gitignore /content/model/.gitignore
!mv /content/README.md /content/model/README.md
!mv /content/config.json  /content/model/config.json
!mv /content/generation_config.json  /content/model/generation_config.json
!mv /content/pytorch_model.bin  /content/model/pytorch_model.bin
!mv /content/special_tokens_map.json  /content/model/special_tokens_map.json
!mv /content/spiece.model  /content/model/spiece.model
!mv /content/tokenizer.json  /content/model/tokenizer.json
!mv /content/tokenizer_config.json  /content/model/tokenizer_config.json
!mv /content/training_args.bin  /content/model/training_args.bin

## Import libraries

In [ ]:
################## PROCEED WITH TEXT SPLITTER #########################
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.create_documents([testo.value])


In [4]:
################## PROCEED WITH TEXT SPLITTER #########################
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 800,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.create_documents([testo.value])

checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
base_model = T5ForConditionalGeneration.from_pretrained(checkpoint,
                                                    device_map='auto',
                                                    torch_dtype=torch.float32)
console.print("[yellow bold] Inizializing pipeline")
pipe_sum = pipeline('summarization', 
                    model = base_model,
                    tokenizer = tokenizer,
                    max_length = 400, 
                    min_length = 25
                    )
start = datetime.datetime.now() #not used now but useful
# taking from ipyWidget textarea content
with console.status('Running Summarization pipeline...', spinner='arrow3'):
  result = pipe_sum(testo.value)
  stop = datetime.datetime.now() #not used now but useful  
  delta = stop-start  
  print(Panel(result[0]['summary_text'], title='AI Summarization'))
  console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing pipeline

Token indices sequence length is longer than the specified maximum sequence length for this model (3172 > 512). Running this sequence through the model will result in indexing errors


Output()

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The article discusses how the era of large language models is over and how the hype surrounding them is causing │
│ a major bubble in the AI space. The article also discusses the importance of data processing and curation in    │
│ implementing large models, as well as the need for companies to invest in improving their data transformation   │
│ processes. The author asks for an anonymous testimonial to help them grow.                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:02:33.018731

With shorter version of the text it seems good

In [9]:
# This text is already with limited number of tokens...
# and can run directly in the pipeline.
# But if your text is longer you will get alarm from tokenizer
#
#Token indices sequence length is longer than the specified maximum sequence length for this model (3103 > 512). 
#Running this sequence through the model will result in indexing errors
# 598 words, 3500 charachters
#################################################################################################################
copied_text = """
Title: Dear Sam Altman- There was never an era of making models bigger  Recently, the internet caught fire with a particular admission from Sam Altman- The Era of Large Language Models is over, further progress will not come from making models bigger.  “We’ll make them better in other ways.” In this article, I’m here to argue something that would be considered blasphemy to many people in the AI Space- the age of mindlessly scaling models has never been here. When we look at the data and actually compare the results- it has always been clear that throwing more and more data and increasing parameter size was always doomed to fail- long before we started hitting the scaling limits that GPT-4 is starting to hit. By understanding how we could have foreseen this problem, we can avoid making these mistakes in the future- saving everyone a lot of time, money, and attention.
Seen from this perspective, you should be less excited about these so-called amazing architectures. When it comes to performance- sure they can hit benchmarks- but at what cost? Deploying these models at any kind of scale would have you running out of computing budgets quicker than Haaland breaking goal-scoring records. Don’t forget, the scale that makes these models powerful also makes them extremely expensive to deploy in contexts where you have to make a lot of inferences.
Here’s something that would surprise you if you only read about GPT from online influencers telling you how to get ahead of 99% of people- these models are just not very good. When it comes to practically implementing Large Language Models into systems that are useful, efficient, and safe- these structures fall apart. When it comes to business use cases, these models are often lose to very simple models.
Prior to this ‘bombshell’ admission and GenAI capturing the attention of everyone, there was another trendy buzzword that was moving through the Data Field- ‘Data-Centric AI’. The premise was relatively- we had spent a lot of time building better models and not enough time on improving our data transformation processes. Take a look at this quote from Andrew Ng at an event by MIT-
AI systems need both code and data, and “all that progress in algorithms means it’s actually time to spend more time on the data,” Ng said at the recent EmTech Digital conference hosted by MIT Technology Review.
It’s been well-known that data processing and curation is the most important part of the pipeline since the beginning.  This is why we have seen the move away from bigger models and obscenely big data sets and instead focus on more intelligent design. Intelligent design was explicitly mentioned as one of the reasons Open Source was beating big companies like Google.
Combining this together, here is a question I want to ask you- when exactly did we have an era of scaling up? At what point was it ever a good idea to implement more and more scaling- as opposed to focusing on better data selection, using ensembles/mixture of experts to reduce errors, or constraining the system to handle certain kinds of problems to avoid errors? People have been calling this unsustainable for a long time, way before general-purpose LLMs were a mainstay in ML.
Rather than an insightful statement about the future of AI, Sam Altman’s statement is a face-saving maneuver. The bubble is bursting, the chickens have come home to roost, and this statement is Sam Altman is now scrambling to maintain the facade that these problems are under control.
"""

In [10]:
checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
base_model = T5ForConditionalGeneration.from_pretrained(checkpoint,
                                                    device_map='auto',
                                                    torch_dtype=torch.float32)
console.print("[yellow bold] Inizializing pipeline")
pipe_sum = pipeline('summarization', 
                    model = base_model,
                    tokenizer = tokenizer,
                    max_length = 550, 
                    min_length = 25
                    )
start = datetime.datetime.now() #not used now but useful
# taking from ipyWidget textarea content
with console.status('Running Summarization pipeline...', spinner='arrow3'):
  result = pipe_sum(copied_text)
  stop = datetime.datetime.now() #not used now but useful  
  delta = stop-start  
  print(Panel(result[0]['summary_text'], title='AI Summarization'))
  console.print(f"[red bold]Summarization completed in {delta}")

 Inizializing pipeline

Token indices sequence length is longer than the specified maximum sequence length for this model (774 > 512). Running this sequence through the model will result in indexing errors


Output()

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ Sam Altman believes that the era of large language models is over and that further progress will not come from  │
│ making models bigger. He argues that throwing more and more data and increasing parameter size was always       │
│ doomed to fail and that the scale that makes these models powerful also makes them extremely expensive to       │
│ deploy in contexts where you have to make a lot of inferences. The article also discusses the importance of     │
│ data processing and curation in AI systems, and the move away from bigger models and obscenely big data sets to │
│ focus on more intelligent design. The author questions when exactly it was a good idea to implement more        │
│ scaling and focusing on better data selection, using ensembles/mixture of experts to reduce errors, or          │
│ constraining the system to handle certain kinds of problems to avoid errors.                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:00:54.992720

> Time to test the results if we club chucks summarized into a final<br>
summarization



---



---



---



## Iterate on the chunks, summarize one by one, and then summarize the summary

In [ ]:
# Function that return the Summarized text
def AI_chunk_Summary(checkpoint, text):
    """
    checkpoint is in the format of relative path
    example:  checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
    text it is either a long string or a input long string or a loaded document into string

    """
    #checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
    checkpoint = checkpoint
    tokenizer = T5Tokenizer.from_pretrained(checkpoint)
    base_model = T5ForConditionalGeneration.from_pretrained(checkpoint,
                                                        device_map='auto',
                                                        torch_dtype=torch.float32)
    pipe_sum = pipeline('summarization', 
                        model = base_model,
                        tokenizer = tokenizer,
                        max_length = 250, 
                        min_length = 25
                        )
    result = pipe_sum(text)
    sum_text = result[0]['summary_text']
    return sum_text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1200,
    chunk_overlap  = 20,
    length_function = len,
)
#texts = text_splitter.create_documents([testo.value]) #valid to create Document Object
texts = text_splitter.split_text(testo.value)
# with testo.value is giving 33 chunks...

In [ ]:
len(texts)

14

In [ ]:
small_sum = AI_chunk_Summary(chkpoint, texts[0])
console.print (small_sum)
type(small_sum)

The article discusses how to leverage LLMs on a domain-specific knowledge base by using Retrieval-Augmented 
Generation to talk to your data. The article also mentions OpenAI's GPT-4 and Meta's LLamA as examples of LLM 
models that were trained on publicly available data.

str

In [ ]:
final_text = ''
chkpoint = '/content/model/'
start = datetime.datetime.now() #not used now but useful
for i in range(0,len(texts)): 
  small_sum = AI_chunk_Summary(chkpoint, texts[i])
  final_text = final_text + small_sum
Summary = AI_chunk_Summary(chkpoint, final_text)
stop = datetime.datetime.now() #not used now but useful  
delta = stop-start
print(Panel(Summary, title='AI Summarization'))  
console.print(f"[red bold]Summarization completed in {delta}")


Your max_length is set to 250, but your input_length is only 234. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=117)
Your max_length is set to 250, but your input_length is only 244. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=122)
Your max_length is set to 250, but your input_length is only 220. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=110)
Your max_length is set to 250, but your input_length is only 221. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The article discusses how to leverage LLMs on a domain-specific knowledge base by using Retrieval-Augmented     │
│ Generation (RAG), which is a semantic search engine that uses vectorial representations of document sections to │
│ determine the meaning stored in each section. Fine-tuned LLM can be used to handle specific jargon and          │
│ incorporate it into the RAG architecture to reap the combined benefits. The main challenges with setting up a   │
│ successful RAG solution include ensuring the quality of the generative AI, attracting and retaining a large     │
│ number of users, managing the traction of the system, and balancing the benefits of RAG with the potential for  │
│ market growth. Proper setup takes time and attention, and selecting the right LLM is important for cost and     │
│ data management. LLMOps can support government staff in answering parliamentary questions more easily, with     │
│ specific knowledge of a moderate database of knowledge articles that contain useful information that is not     │
│ easily found on the world-wide web.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:05:02.822489

## Using LangChain CHAIN

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel, T5Tokenizer, T5Model
from transformers import T5ForConditionalGeneration, pipeline
import torch
import rich
from rich.console import Console
from rich.panel import Panel
from rich import print
import ssl
import requests
import datetime
import ipywidgets as widgets
from IPython.display import Image, display
from tqdm import tqdm

########### IMPORT HFACE PIPELINE FROM LANGCHAIN AND SUMMARIZE CHAIN ################
from langchain.llms import HuggingFacePipeline 
from langchain.chains.summarize import load_summarize_chain 

############### DEFINING THE LAMINI MODEL CHACKPOINT IN LANGCHAIN #################
checkpoint = "/content/model/"  #it is actually LaMini-Flan-T5-248M   #tested fine
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
base_model = T5ForConditionalGeneration.from_pretrained(checkpoint,
                                                    device_map='auto',
                                                    torch_dtype=torch.float32)

################## HUGGINGFACEPIPELINE DETAILS  ###################
# To use, you should have the transformers python package installed.
# Only supports text-generation, text2text-generation and summarization for now.
llm = HuggingFacePipeline.from_model_id(model_id=checkpoint, 
                                        task = 'text2text-generation', 
                                        model_kwargs={"temperature":0, "max_length":512})

########## INITIALIZE RICH CONSOLE  ##################
console = Console()

################## PREPARE WIDGET #########################
testo = widgets.Textarea(
    value='',
    placeholder=f'Paste here the text you want to summarize',
    description=f'TEXT:',
    disabled=False,
    rows = 20,   
    layout = widgets.Layout( width='75%')
)
display(testo)

Textarea(value='', description='TEXT:', layout=Layout(width='75%'), placeholder='Paste here the text you want …

In [20]:
################## TEXT SPLITTER FUNCTION #################
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 460,
    chunk_overlap  = 40,
    length_function = len,
)
texts = text_splitter.create_documents([testo.value])

########### RUN THE SUMMARIZE CHAIN ######################

summaries = []
start = datetime.datetime.now() #not used now but useful
console.print("[yellow bold] Inizializing Summarize Chain")
chain = load_summarize_chain(llm, chain_type="map_reduce")
with console.status('Running Summarization chain...', spinner='point'):
  summary = chain.run(texts)
  console.print("Summary for: User input text")
  console.print(f"with LaMini-Flan-T5-248M and LangChain \n")
  summaries.append(summary)
  stop = datetime.datetime.now() #not used now but useful  
  delta = stop-start  
  print(Panel(summary, title='AI Summarization'))  
  console.print(f"[red bold]Summarization completed in {delta}")


Output()

Token indices sequence length is longer than the specified maximum sequence length for this model (1808 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1860 > 512). Running this sequence through the model will result in indexing errors


Summary for: User input text

with LaMini-Flan-T5-248M and LangChain

╭─────────────────────────────────────────────── AI Summarization ────────────────────────────────────────────────╮
│ The article discusses the lack of mindlessly scaling models in AI and the importance of understanding how to    │
│ scale models before scaling them. The article also mentions the hype surrounding certain models and suggests    │
│ that it's time to get into the subject of image source saturation of benchmarks. The article also mentions the  │
│ importance of data processing and curation in the pipeline, and suggests that people should spend more time on  │
│ data. The article also mentions the streetlight effect, a type of observational bias where people focus on      │
│ funding or copying what is already being done, resulting in a mindless scaling model size without attention     │
│ given to other factors. The article concludes by highlighting the need for more scaling and highlighting the    │
│ importance of building better models and improving data transformation processes.                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Summarization completed in 0:04:12.744316

### The 🥇 *text2text* generation pipeline with   🦜🔗 LangChain win both in terms of time ⏲ and in terms of 👍 quality